# LLM ModelTest #2
#### w/ Chroma

In [12]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv
import os

# API KEY 정보로드
load_dotenv()
GEMINI_API_KEY = os.getenv('API_KEY_GEMINI')

In [13]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("lgdx_team2")

LangSmith 추적을 시작합니다.
[프로젝트명]
lgdx_team2


In [3]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders.csv_loader import CSVLoader

from langchain_community.vectorstores import Chroma

from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

import pandas as pd

# 랭체인 환경 설정
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI

In [5]:
### 01. CSV 파일에서 문서 로드 ###
loader = CSVLoader('../../data/movie_4000_preprocessed.csv', encoding='utf8')
docs = loader.load()
print(f"문서의 수: {len(docs)}")

### 02. pandas로 데이터프레임 칼럼명 가져오기
csv_path = '../../data/movie_4000_preprocessed.csv'
df = pd.read_csv(csv_path, encoding='utf8')
colnames = df.columns

문서의 수: 3901


In [6]:
### 03. 메타데이터 추가 ###
docs = []
for _, row in df.iterrows():
  # 필요한 메타데이터 설정
  metadata = {
    'title': row['movie_title'],
    'genre': row['genre']
  }
  # 각 행의 데이터를 문서로 변환
  doc = Document(
    page_content=str(row.to_dict()),
    metadata=metadata
  )
  docs.append(doc)

print(f"문서의 수: {len(docs)}")
print('[메타데이터 예시]\n', docs[100].metadata)

문서의 수: 3901
[메타데이터 예시]
 {'title': '아름답다', 'genre': '드라마'}


In [7]:
### 04. 데이터 청크 나누기 ###
text_splitter = RecursiveCharacterTextSplitter(
  chunk_size=1090, chunk_overlap=0
)
splits = text_splitter.split_documents(docs)
print("split된 문서의 수:", len(splits))

split된 문서의 수: 3901


In [8]:
### 05. 임베딩 모델 생성
# https://huggingface.co/ibm-granite/granite-embedding-278m-multilingual
embeddings = HuggingFaceEmbeddings(model_name='ibm-granite/granite-embedding-278m-multilingual')

C:\Users\USER\AppData\Local\Temp\ipykernel_11492\3636801640.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name='ibm-granite/granite-embedding-278m-multilingual')
C:\workspaces\LGDXteam2\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [39]:
### 06. 벡터스토어 생성 ###
DB_PATH = '../movie_4000_vectorstore_2'

## Chroma
# vectorstore = Chroma.from_documents(
#   documents=splits,
#   embedding=embeddings,
#   persist_directory=DB_PATH,
# )
# print("벡터스토어 저장 완료!")

벡터스토어 저장 완료!


In [10]:
# Chroma 벡터스토어 로드
vectorstore = Chroma(persist_directory="../movie_4000_vectorstore_2", embedding_function=embeddings)

In [11]:
# 검색 테스트
results = vectorstore.similarity_search_with_relevance_scores(
    query="미국 배경의 액션 영화",
    k=5)

# 유사도 점수 높은 순으로 정렬
sorted_results = sorted(results, key=lambda x: x[1], reverse=True)

# 결과 출력
for doc, score in sorted_results:
    print(f"🎬 제목: {doc.metadata.get('title', 'N/A')}")
    print(f"🎭 장르: {doc.metadata.get('genre', 'N/A')}")
    print(f"📄 요약: {doc.page_content[:200]}...")  # 요약 일부만 출력
    print(f"⭐ 유사도 점수: {score:.4f}")  # 소수점 4자리까지 출력
    print("=" * 80)  # 구분선

🎬 제목: 학살기관
🎭 장르: 애니메이션
📄 요약: {'movie_title': '학살기관', 'genre': '애니메이션', 'summary': '테러의 위험에 계속해서 노출되고 있는 미국은 그 공포에 대항하기 위해 철저한 정보 관리 시스템을 구축하고 있다.  한편, 미국 이외의 세계 각지에서는 분쟁의 격화게 계속되고 있다.  세계의 분쟁지를 돌아다니는 미군 특수부대 클라비스 셰퍼드 대위에게 수수께기로 둘...
⭐ 유사도 점수: 0.6420
🎬 제목: 아메리칸 시빌 워
🎭 장르: 공포/스릴러
📄 요약: {'movie_title': '아메리칸 시빌 워', 'genre': '공포/스릴러', 'summary': '미국에서 벌어지는 우익과 좌익 간의 내전을 배경으로, 보수의 성향을 가진 어느 민주주의 소모임의 구성원들은 정부의 탄압을 피해 외딴 산장에 납치로 위장된 채 도착한다. 그들은 종전 이후 산장을 빠져나갈 생각을 하며 하루하루 버티는데, 시골 산장지기의 ...
⭐ 유사도 점수: 0.6297
🎬 제목: 원샷
🎭 장르: 액션/어드벤쳐
📄 요약: {'movie_title': '원샷', 'genre': '액션/어드벤쳐', 'summary': '"단 원테이크의 리얼타임 액션 액션의 패러다임이 바뀐다." 악명 높은 테러리스트들이 수감된 일급비밀의 섬. 예고된 워싱턴 테러의 배후를 아는 `놈`을 이송하려 네이비 씰과 CIA 분석관이 도착한다. 그때 `놈` 탈옥시키려는 테러단의 공격이 시작되는데…  단 한순...
⭐ 유사도 점수: 0.6207
🎬 제목: 엽표행동
🎭 장르: 액션/어드벤쳐
📄 요약: {'movie_title': '엽표행동', 'genre': '액션/어드벤쳐', 'summary': '홍콩 경시청과 미국 CIA 요원이 국제 거물 동양을 미국으로 호송하는 도중 폭력조직으로부터 습격을 받는다. 배신한 동양으로부터 전자칩을 빼내려고 폭력단의 두목 차부가 한낮의 습격전을 벌인 것이다. 거물급 스파이 동양을 호송하기 위해 홍콩에 온 CIA 아용, ...
⭐ 유사도 점수:

<br><hr>

In [58]:
### 03. 랭체인 구성 ###
# 검색기 생성
retriever = vectorstore.as_retriever(
    search_type="mmr",   
    search_kwargs={"k": 20,              # 반환할 문서 수 (default: 4)
                   "fetch_k": 50,       # MMR 알고리즘에 전달할 문서 수
                   "lambda_mult": 0.5,    # 결과 다양성 조절 (default: 0.5),
                   }
)

# 프롬프트 템플릿 설정
template = """
You are a movie-recommendation chatbot.
You must only answer based on the given context.
Do not generate answers that are not directly supported by the context.

[Context]:
{retrieved_context}

[Question]:
{query}

[Answer]:
"""
prompt = ChatPromptTemplate.from_template(template)

# Google Gemini 모델 생성
def load_gemini(system_instruction):
    model = ChatGoogleGenerativeAI(
        model='gemini-1.5-flash',
        temperature=0.3,
        max_tokens=5000,
        system_instruction=system_instruction,
        api_key=GEMINI_API_KEY
    )
    print(">>>>>>> model loaded...")
    return model

system_instruction = """you are a movie-recommendation chatbot. you must answer based on given data."""
llm = load_gemini(system_instruction)

>>>>>>> model loaded...


In [59]:
# langchain 체인 구성
rag_chain = (
  {"query":RunnablePassthrough(),
    "retrieved_context": retriever,
  }
  # question(사용자의 질문) 기반으로 연관성이 높은 문서 retriever 수행 >> format_docs로 문서를 하나로 만듦
  | prompt               # 하나로 만든 문서를 prompt에 넘겨주고
  | llm                  # llm이 원하는 답변을 만듦
  | StrOutputParser()
)

In [64]:
### 04. 모델 테스트 ###
user_query='코미디면서 호러 장르의 영화를 추천해줘'
response = rag_chain.invoke(user_query)
print(response)

쇼미더고스트와 좀비 사용 설명서를 추천합니다.

